[View in Colaboratory](https://colab.research.google.com/github/gopal2812/mlblr/blob/master/datawithlowdropout.ipynb)

In [6]:
!pip install -q keras
import keras

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

batch_size = 128
num_classes = 10
epochs = 50
l = 24
num_filter = 64
compression = 0.5
dropout_rate = 0.4

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp
  
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg
  
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output
  
num_filter = 64
dropout_rate = 0.5
l = 6
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

model = Model(inputs=[input], outputs=[output])
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_85 (Conv2D)              (None, 32, 32, 64)   1728        input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_85 (BatchNo (None, 32, 32, 64)   256         conv2d_85[0][0]                  
__________________________________________________________________________________________________
activation_85 (Activation)      (None, 32, 32, 64)   0           batch_normalization_85[0][0]     
__________________________________________________________________________________________________
conv2d_86 

50000/50000 [==============================] - 195s 4ms/step - loss: 1.6537 - acc: 0.3822 - val_loss: 3.0089 - val_acc: 0.2663
Epoch 2/50
38272/50000 [=====================>........] - ETA: 40s - loss: 1.2721 - acc: 0.5351

50000/50000 [==============================] - 187s 4ms/step - loss: 1.2508 - acc: 0.5441 - val_loss: 1.6024 - val_acc: 0.5351
Epoch 3/50
50000/50000 [==============================] - 187s 4ms/step - loss: 1.0510 - acc: 0.6250 - val_loss: 1.2945 - val_acc: 0.6268
Epoch 4/50
 1536/50000 [..............................] - ETA: 2:49 - loss: 0.9899 - acc: 0.6471

50000/50000 [==============================] - 187s 4ms/step - loss: 0.9503 - acc: 0.6592 - val_loss: 1.9197 - val_acc: 0.5306
Epoch 5/50
38784/50000 [======================>.......] - ETA: 39s - loss: 0.8985 - acc: 0.6809

50000/50000 [==============================] - 187s 4ms/step - loss: 0.8892 - acc: 0.6832 - val_loss: 1.4446 - val_acc: 0.6065
Epoch 6/50
50000/50000 [==============================] - 187s 4ms/step - loss: 0.8301 - acc: 0.7044 - val_loss: 2.6600 - val_acc: 0.4353
Epoch 7/50
 1664/50000 [..............................] - ETA: 2:48 - loss: 0.8235 - acc: 0.7085

...

50000/50000 [==============================] - 187s 4ms/step - loss: 0.3458 - acc: 0.8784 - val_loss: 0.6939 - val_acc: 0.8266
Epoch 45/50
50000/50000 [==============================] - 187s 4ms/step - loss: 0.3456 - acc: 0.8784 - val_loss: 0.8490 - val_acc: 0.7973
Epoch 46/50
 1536/50000 [..............................] - ETA: 2:48 - loss: 0.3187 - acc: 0.8867

50000/50000 [==============================] - 187s 4ms/step - loss: 0.3383 - acc: 0.8831 - val_loss: 0.6863 - val_acc: 0.8417
Epoch 47/50
38656/50000 [======================>.......] - ETA: 39s - loss: 0.3328 - acc: 0.8831

50000/50000 [==============================] - 187s 4ms/step - loss: 0.3342 - acc: 0.8820 - val_loss: 0.5864 - val_acc: 0.8555
Epoch 48/50
50000/50000 [==============================] - 187s 4ms/step - loss: 0.3320 - acc: 0.8829 - val_loss: 0.9751 - val_acc: 0.7704
Epoch 49/50
 1536/50000 [..............................] - ETA: 2:48 - loss: 0.3123 - acc: 0.8945

50000/50000 [==============================] - 187s 4ms/step - loss: 0.3272 - acc: 0.8866 - val_loss: 0.9012 - val_acc: 0.8046
Epoch 50/50
38656/50000 [======================>.......] - ETA: 39s - loss: 0.3183 - acc: 0.8880

10000/10000 [==============================] - 17s 2ms/step
('Test loss:', 0.6260204187035561)
('Test accuracy:', 0.8506)


In [7]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

from google.colab import files
files.download('DNST_model.h5')

Saved model to disk
